In [4]:
'''
    TEMPLATE FOR MACHINE LEARNING HOMEWORK
    AUTHOR Eric Eaton, Chris Clingerman
'''
import math
import numpy as np
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.metrics import accuracy_score

numOfFoldsPerTrial = 10

def evaluatePerformance(numTrials=100):
    '''
    Evaluate the performance of decision trees,
    averaged over 1,000 trials of 10-fold cross validation
    
    Return:
      a matrix giving the performance that will contain the following entries:
      stats[0,0] = mean accuracy of decision tree
      stats[0,1] = std deviation of decision tree accuracy
      stats[1,0] = mean accuracy of decision stump
      stats[1,1] = std deviation of decision stump
      stats[2,0] = mean accuracy of 3-level decision tree
      stats[2,1] = std deviation of 3-level decision tree
      
    ** Note that your implementation must follow this API**
    '''
    
    # Load Data
    filename = 'data/SPECTF.dat'
    data = np.loadtxt(filename, delimiter=',')
    X = data[:, 1:]
    y = np.array([data[:, 0]]).T
    n,d = X.shape

    # create list to hold data
    treeAccuracies = []
    stumpAccuracies = []
    dt3Accuracies = []
    train_size = []
    tree1 = np.zeros((10, numTrials*numOfFoldsPerTrial))
    tree_stump = np.zeros((10, numTrials*numOfFoldsPerTrial))
    tree3 = np.zeros((10, numTrials*numOfFoldsPerTrial))

    # perform 100 trials
    for x in range(0, numTrials):
        # shuffle the data
        idx = np.arange(n)
        np.random.seed(13)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]

        # split the data randomly into 10 folds
        folds = []
        intervalDivider = int(len(X)/numOfFoldsPerTrial)
        for fold in range(0, numOfFoldsPerTrial):
            # designate a new testing range
            Xtest = X[fold * intervalDivider:(fold + 1) * intervalDivider,:]
            ytest = y[fold * intervalDivider:(fold + 1) * intervalDivider,:]
            Xtrain = X[:(fold * intervalDivider),:]
            ytrain = y[:(fold * intervalDivider),:]
            Xtrain = Xtrain.tolist()
            ytrain = ytrain.tolist()
            total_accuracy_regular_tree = 0
            total_accuracy_stump = 0
            total_accuracy_DT3 = 0

            # complete the training data set so that it contains all
            # data except for the current test fold
            for dataRow in range((fold + 1) * intervalDivider, len(X)):
                Xtrain.append(X[dataRow])
                ytrain.append(y[dataRow])
            
            total_train = len(ytrain)
            total_test = len(ytest)
            for percent in range(10,101,10):
                train_size = math.floor(total_train*percent/100)
                X_train = Xtrain[:train_size] 
                y_train = ytrain[:train_size]
                 # train the decision tree
                clf = tree.DecisionTreeClassifier()
                clf = clf.fit(X_train,y_train)
                 # train the 1-level decision tree
                oneLevel = tree.DecisionTreeClassifier(max_depth=1)
                oneLevel = oneLevel.fit(X_train,y_train)

                # train the 3-level decision tree
                threeLevel = tree.DecisionTreeClassifier(max_depth=3)
                threeLevel = threeLevel.fit(X_train,y_train)
                # output predictions on the remaining data
                y_pred_tree = clf.predict(Xtest)
                y_pred_stump = oneLevel.predict(Xtest)
                y_pred_dt3 = threeLevel.predict(Xtest)
                # compute the training accuracy of the model and save to the 
                # list of all accuracies
                if percent == 100:
                
                    treeAccuracies.append(accuracy_score(ytest, y_pred_tree))
                    stumpAccuracies.append(accuracy_score(ytest, y_pred_stump))
                    dt3Accuracies.append(accuracy_score(ytest, y_pred_dt3))

                idx_percent = math.floor(percent/10-1)
                idx_trial_fold = x * numOfFoldsPerTrial+ fold
                tree1[idx_percent][idx_trial_fold] = accuracy_score(ytest, y_pred_tree)
                tree_stump[idx_percent][idx_trial_fold] = accuracy_score(ytest, y_pred_stump)
                tree3[idx_percent][idx_trial_fold] = accuracy_score(ytest, y_pred_dt3)
   
    
    meanDecisionTreeAccuracy = np.mean(treeAccuracies)
    stddevDecisionTreeAccuracy = np.std(treeAccuracies)
    meanDecisionStumpAccuracy = np.mean(stumpAccuracies)
    stddevDecisionStumpAccuracy = np.std(stumpAccuracies)
    meanDT3Accuracy = np.mean(dt3Accuracies)
    stddevDT3Accuracy = np.std(dt3Accuracies)
    ntrain = np.arange(10,101,10)
     # Update these statistics based on the results of your experiment
    #plot
    plt.plot(ntrain,tree1, label = 'Tree1')
    plt.plot(ntrain,tree_stump, label ='Tree stump')
    plt.plot(ntrain,tree3, label = 'Tree3')
    plt.title('Learning curve')
    plt.xlabel('Training size')
    plt.ylabel('Accuracy')
    plt.legend(loc = 'best')

    # make certain that the return value matches the API specification
    stats = np.zeros((3,2))
    stats[0,0] = meanDecisionTreeAccuracy
    stats[0,1] = stddevDecisionTreeAccuracy
    stats[1,0] = meanDecisionStumpAccuracy
    stats[1,1] = stddevDecisionStumpAccuracy
    stats[2,0] = meanDT3Accuracy
    stats[2,1] = stddevDT3Accuracy
    return stats



# Do not modify from HERE...
if __name__ == "__main__":
    
    stats = evaluatePerformance()
    print ("Decision Tree Accuracy = ", stats[0,0], " (", stats[0,1], ")")
    print ("Decision Stump Accuracy = ", stats[1,0], " (", stats[1,1], ")")
    print ("3-level Decision Tree = ", stats[2,0], " (", stats[2,1], ")")
# ...to HERE.

TypeError: slice indices must be integers or None or have an __index__ method